In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,datetime,os

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Conv1D,Conv2D,BatchNormalization,Dropout,Activation,Add,Concatenate,Bidirectional,Conv2DTranspose,Flatten

from sklearn.manifold import TSNE

import gym

warnings.filterwarnings('ignore')
sns.set()

C:\Users\Jan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def discount_rewards(R,discount_factor=0.98):
    RR = []
    gamma = [discount_factor**(R.shape[0]-k-1) for k in range(R.shape[0])]
    for k in range(R.shape[0]):
        RR.append(R[k]*gamma[k])
    RR = np.stack(RR,axis=0)
    return RR

In [141]:
learning_rate = 1

tf.reset_default_graph()

tf_state = tf.placeholder(dtype=tf.float32,shape=[None,4])
tf_rewards = tf.placeholder(dtype=tf.float32,shape=[None,1])

M0 = tf.Variable(dtype=tf.float32,initial_value=tf.random_normal([4,3]))
M1 = tf.Variable(dtype=tf.float32,initial_value=tf.random_normal([3,3]))
M2 = tf.Variable(dtype=tf.float32,initial_value=tf.random_normal([3,3]))
M3 = tf.Variable(dtype=tf.float32,initial_value=tf.random_normal([3,2]))

b0 = tf.Variable(dtype=tf.float32,initial_value=tf.random_normal([3]))
b1 = tf.Variable(dtype=tf.float32,initial_value=tf.random_normal([3]))
b2 = tf.Variable(dtype=tf.float32,initial_value=tf.random_normal([3]))
b3 = tf.Variable(dtype=tf.float32,initial_value=tf.random_normal([2]))

x0 = tf.nn.relu(tf.add(tf.matmul(tf_state,M0),b0))
x1 = tf.nn.relu(tf.add(tf.matmul(x0,M1),b1))
x2 = tf.nn.relu(tf.add(tf.matmul(x1,M2),b2))
policy = tf.nn.softmax(tf.add(tf.matmul(x2,M3),b3))

action_sugg = tf.argmax(policy,axis=1)

log_p = tf.log(policy)

log_grad_M0 = tf.gradients(log_p,M0)
log_grad_M1 = tf.gradients(log_p,M1)
log_grad_M2 = tf.gradients(log_p,M2)
log_grad_M3 = tf.gradients(log_p,M3)

log_grad_b0 = tf.gradients(log_p,b0)
log_grad_b1 = tf.gradients(log_p,b1)
log_grad_b2 = tf.gradients(log_p,b2)
log_grad_b3 = tf.gradients(log_p,b3)

updateM0 = M0.assign_add(learning_rate*tf.multiply(tf_rewards,log_grad_M0[0]))
updateM1 = M1.assign_add(learning_rate*tf.multiply(tf_rewards,log_grad_M1[0]))
updateM2 = M2.assign_add(learning_rate*tf.multiply(tf_rewards,log_grad_M2[0]))
updateM3 = M3.assign_add(learning_rate*tf.multiply(tf_rewards,log_grad_M3[0]))
updateb0 = b0.assign_add(learning_rate*tf.multiply(tf_rewards[0],log_grad_b0[0]))
updateb1 = b1.assign_add(learning_rate*tf.multiply(tf_rewards[0],log_grad_b1[0]))
updateb2 = b2.assign_add(learning_rate*tf.multiply(tf_rewards[0],log_grad_b2[0]))
updateb3 = b3.assign_add(learning_rate*tf.multiply(tf_rewards[0],log_grad_b3[0]))

In [142]:
max_epochs = 250
remove_n_last = 3
render_env = False

In [143]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

env = gym.make('CartPole-v0')

policies = []

epoch = 0
while epoch<=max_epochs:
    
    state = env.reset()
    
    states = []
    actions = []
    rewards = []

    done = False
    while not done:
        if render_env: env.render()
        states.append(state)
        action,nn_policy = sess.run([action_sugg,policy],feed_dict={
            tf_state:state.reshape(-1,4)
        })
        action = action[0]
        actions.append(action)
        state,reward,done,info = env.step(action)
        rewards.append(reward)
        
        
    policies.append(nn_policy)   
    epoch += 1
    
    states = np.stack(states[:-remove_n_last],axis=0)
    actions = np.stack(actions[:-remove_n_last],axis=0)
    rewards = discount_rewards(np.stack(rewards[:-remove_n_last],axis=0))
    
    for ustate,ureward in zip(states,rewards):
        
        _ = sess.run([updateM0,updateM1,updateM2,updateM3,updateb0,updateb1,updateb2,updateb3],feed_dict={
            tf_state:ustate.reshape(-1,4),
            tf_rewards:ureward.reshape(-1,1)
        })

env.close()
sess.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [144]:
policies

[array([[0.13222878, 0.8677712 ]], dtype=float32),
 array([[0.7014763 , 0.29852372]], dtype=float32),
 array([[0.48129994, 0.5187001 ]], dtype=float32),
 array([[0.553166  , 0.44683406]], dtype=float32),
 array([[0.4782848, 0.5217152]], dtype=float32),
 array([[0.5389871 , 0.46101287]], dtype=float32),
 array([[0.48660254, 0.51339746]], dtype=float32),
 array([[0.55252665, 0.44747335]], dtype=float32),
 array([[0.47662604, 0.523374  ]], dtype=float32),
 array([[0.5430688, 0.4569312]], dtype=float32),
 array([[0.48084143, 0.51915854]], dtype=float32),
 array([[0.538069  , 0.46193093]], dtype=float32),
 array([[0.48286682, 0.5171332 ]], dtype=float32),
 array([[0.52524704, 0.47475296]], dtype=float32),
 array([[0.49118963, 0.50881034]], dtype=float32),
 array([[0.5212452 , 0.47875485]], dtype=float32),
 array([[0.49313036, 0.5068697 ]], dtype=float32),
 array([[0.52109617, 0.4789038 ]], dtype=float32),
 array([[0.4927096, 0.5072904]], dtype=float32),
 array([[0.51018804, 0.48981193]], dt